Libaries

In [66]:
# import the necessary libraries you need for your analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')


#brew install cmake
#pip install xgboost==1.6.1

RSEED = 42

Reading data

In [67]:
df = pd.read_csv('data/2_data.csv')

Processing data

In [68]:
#replace state --> failed == 0 and successfull == 1
df['state'] = df['state'].replace('failed', 0).replace('successful', 1).astype(int)

#change dtype for duration from object into int
df[['duration', 'duration_days_weeks', 'duration_time']] = df['duration'].str.split(' ', expand=True)
df = df.drop('duration_days_weeks', axis = 1).drop('duration_time', axis = 1)
df.duration = df.duration.astype(int)

XGBOOST

In [69]:
df_final2 = df[['staff_pick','category_sub','duration','description_length', 'name_length', 'goal_usd','state']]
df_final2.description_length.fillna(0, inplace=True)
df_final2['state'] = df_final2['state'].replace({'failed': 0, 'successful': 1})
df_final2 = pd.get_dummies(df_final2, drop_first = True)

y2 = df_final2["state"]
X2 = df_final2.drop("state", axis=1)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, stratify = y2, random_state=RSEED)


In [70]:
col = ['duration','description_length', 'name_length', 'goal_usd']
scaling2 = StandardScaler()
X_train_scaled = scaling2.fit_transform(X_train2[col])
X_test_scaled = scaling2.transform(X_test2[col])

X_train_preprocessed = np.concatenate([X_train_scaled, X_train2.drop(col , axis=1)], axis= 1)
X_test_preprocessed = np.concatenate([X_test_scaled, X_test2.drop(col , axis=1)], axis = 1)

In [71]:
#fitting XGBoost with final params and dropped main category

boost_final = XGBClassifier(learning_rate = 0.5, max_depth = 5, min_child_weight = 50, subsample = 0.9)

boost_final.fit(X_train_preprocessed, y_train2)

y_pred_final = boost_final.predict(X_test_preprocessed)

print("Performance on train set: ",boost_final.score(X_train_preprocessed, y_train2))
print("Performance on test set: ",boost_final.score(X_test_preprocessed, y_test2))


print(classification_report(y_test2,y_pred_final))

Performance on train set:  0.8065554626225007
Performance on test set:  0.8
              precision    recall  f1-score   support

           0       0.76      0.80      0.78     18550
           1       0.84      0.80      0.82     23695

    accuracy                           0.80     42245
   macro avg       0.80      0.80      0.80     42245
weighted avg       0.80      0.80      0.80     42245



In [72]:
#staff_pick','category_sub','duration','description_length', 'name_length', 'goal_usd','state'
def predict_funct(staffpick, category_sub, duration, description_length, name_length, goal_usd):
    """_summary_

    Args:
        staffpick (_type_): _description_
        category_sub (_type_): _description_
        duration (_type_): _description_
        description_length (_type_): _description_
        name_length (_type_): _description_
        goal_usd (_type_): _description_

    Returns:
        _type_: _description_
    """
    X_kt_scale = scaling2.transform((np.array([duration, description_length, name_length, goal_usd]).reshape(1,-1)))
    X = X_train2.head(1)
    X = pd.DataFrame(0, index=np.arange(len(X_bp)), columns=X_bp.columns)

    X.staff_pick =  staffpick
    X[category_sub] = 1
    X_kickturtle = np.concatenate([X_kt_scale, X.drop(['duration', 'description_length', 'name_length', 'goal_usd'],axis=1)],axis=1)
    y_kt = boost_final.predict(X_kickturtle)


    return print(y_kt)

In [78]:
predict_funct(False,'category_sub_webcomics',1,100,100,1000000000000)  #Example how to call the function
predict_funct(True,'category_sub_Misc',30,100,100,1000) 

[0]
[1]
